In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import Xception
from keras.applications.xception import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 35534421
EPOCHS = 150
size = 71
batchsize = 256
lw = 6
channel = 1
STEPS = TOT / EPOCHS / batchsize
NCATS = 340
mixup = 0
center = True
optimizer = 'adam'
cb = 'raw'

In [3]:
K.clear_session()
model = Xception(input_shape=(size, size, 1), weights=None, classes=NCATS)

if optimizer == 'sgd':
    opt = SGD(0.002, momentum=0.9, nesterov=True)
    opt = NormalizedOptimizer(opt, normalization='l2')
elif optimizer == 'adam':
    opt = Adam(lr=0.002)

model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 71, 71, 1)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 35, 35, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 35, 35, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 35, 35, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [4]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input, 
                               channel=channel, center = center)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [5]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 71, 71, 1) (34000, 340)
Validation array memory 0.64 GB


In [6]:
train_datagen = image_generator_xd(df_path = '../input/train_all_balance.csv', 
                                   size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel, mixup=mixup, center=center)

In [7]:
model_prefix = 'xception{}_lw{}_balance_mix{}_{}_{}'.format(size, lw, mixup, optimizer, cb)
if center:
    model_prefix += '_center'
print(model_prefix)
check_path = "./models/{}.model".format(model_prefix)
fold = 9
if cb == 'raw':
    callbks = [
        ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=5,
                          min_delta=0.005, mode='max', cooldown=3, verbose=1),
        callbacks.ModelCheckpoint(check_path,
                                    monitor='val_top_3_accuracy', 
                                    mode = 'max', save_best_only=True, verbose=1),
        callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
        EarlyStopping(monitor='val_top_3_accuracy', mode='max',patience=20, verbose=1)
    ]
elif cb == 'snap':
    snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS,
                                       nb_snapshots=1,
                                       size = size,
                                       init_lr=1e-3, 
                                       fold=fold)

    callbks = snapshot.get_callbacks(model_prefix = model_prefix)

xception71_lw6_balance_mix0_adam_raw_center


In [8]:
# model.load_weights('models/xception_raw_deep71.model')

In [ ]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/150
926/925 [==============================] - 413s 446ms/step - loss: 2.5366 - categorical_crossentropy: 2.5366 - categorical_accuracy: 0.4200 - top_3_accuracy: 0.6183 - val_loss: 2.4776 - val_categorical_crossentropy: 2.4776 - val_categorical_accuracy: 0.4510 - val_top_3_accuracy: 0.6701

Epoch 00001: val_top_3_accuracy improved from -inf to 0.67015, saving model to ./models/xception71_lw6_balance_mix0_adam_raw_center.model
Epoch 2/150
926/925 [==============================] - 415s 449ms/step - loss: 1.6409 - categorical_crossentropy: 1.6409 - categorical_accuracy: 0.5958 - top_3_accuracy: 0.7914 - val_loss: 1.8280 - val_categorical_crossentropy: 1.8280 - val_categorical_accuracy: 0.5649 - val_top_3_accuracy: 0.7676

Epoch 00002: val_top_3_accuracy improved from 0.67015 to 0.76759, saving model to ./models/xception71_lw6_balance_mix0_adam_raw_center.model
Epoch 3/150
926/925 [==============================] - 415s 448ms/step - loss: 1.4335 - categorical_crossentropy: 1.4335 


Epoch 00020: val_top_3_accuracy improved from 0.90244 to 0.90500, saving model to ./models/xception71_lw6_balance_mix0_adam_raw_center.model
Epoch 21/150
926/925 [==============================] - 414s 447ms/step - loss: 0.9025 - categorical_crossentropy: 0.9025 - categorical_accuracy: 0.7679 - top_3_accuracy: 0.9071 - val_loss: 0.9179 - val_categorical_crossentropy: 0.9179 - val_categorical_accuracy: 0.7640 - val_top_3_accuracy: 0.9052

Epoch 00021: val_top_3_accuracy improved from 0.90500 to 0.90524, saving model to ./models/xception71_lw6_balance_mix0_adam_raw_center.model
Epoch 22/150
926/925 [==============================] - 414s 447ms/step - loss: 0.8957 - categorical_crossentropy: 0.8957 - categorical_accuracy: 0.7692 - top_3_accuracy: 0.9087 - val_loss: 0.9166 - val_categorical_crossentropy: 0.9166 - val_categorical_accuracy: 0.7651 - val_top_3_accuracy: 0.9059

Epoch 00022: val_top_3_accuracy improved from 0.90524 to 0.90585, saving model to ./models/xception71_lw6_balance_m

In [ ]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel, center = center)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}


In [ ]:
def doodle_predict(model, model_path, x_test):
    model.load_weights(model_path)

    test_predictions = model.predict(x_test, batch_size=128, verbose=1)
    top3 = preds2catids(test_predictions)
    top3cats = top3.replace(id2cat)
    test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
    submission = test[['key_id', 'word']]

    import kaggle_util
    kaggle_util.save_result(submission,  
                            '../result/{}.csv'.format(model_prefix), 
                            'quickdraw-doodle-recognition', 
                            send=True, index=False)

In [ ]:
doodle_predict(model, check_path, x_test)